In [1]:
%load_ext autoreload
%autoreload 2

import logging
from pathlib import Path
from warnings import simplefilter

import pandas as pd
from dotenv import dotenv_values

from generator_drainage_units import run_generator_order_levels
from generator_drainage_units.utils.network_functions import (
    find_nodes_edges_for_direction,
)

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
logging.basicConfig(level=logging.DEBUG)

In [2]:
# Define case and base_dir

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
waterschap = "Waterschap Vallei & Veluwe"
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
# case_name = "hattemerbroek"
# case_name = "pangelerbeek"
case_name = "Leuvenumse_beek"

# Define waterboard specific variables
range_orde_code_min = 721
range_orde_code_max = 760

In [ ]:
%%time
case_path = Path(base_dir, case_name)
order = run_generator_order_levels(
    path=case_path,
    waterschap=waterschap,
    range_orde_code_min=range_orde_code_min,
    range_orde_code_max=range_orde_code_max,
    water_lines=["hydroobjecten"],
    read_results=True,
    write_results=True,
    create_html_map=False,
    open_html=False
)

In [ ]:
order.outflow_nodes_all

In [ ]:
logging.info(f"   x generate orde levels for all hydroobjects")
order.outflow_nodes_all = order.outflow_nodes_all[order.outflow_nodes_all["orde_nr"]==2].copy()

new_outflow_nodes_all = order.outflow_nodes_all[order.outflow_nodes_all["orde_nr"]==2].copy()
edges_left = order.edges.copy()
nodes_left = order.nodes[order.nodes.nodeID.isin(edges_left.node_end) | order.nodes.nodeID.isin(edges_left.node_start)].copy()
order_nr = 2
order_labels = False if case_name == "vallei_en_veluwe" else True

while not new_outflow_nodes_all.empty and order_nr<10:
    logging.debug(f"    - order {order_nr}: {len(new_outflow_nodes_all)} outflow nodes")
    outflow_nodes = new_outflow_nodes_all.copy()
    new_outflow_nodes_all = None
    edges_order_all = None
    for i_node, outflow_node in outflow_nodes.iterrows():
        print(f" * {i_node+1}/{len(outflow_nodes)}", end="\r")
        nodes_order, edges_order, new_outflow_nodes = find_nodes_edges_for_direction(
            nodes=nodes_left,
            edges=edges_left,
            node_ids=[outflow_node["nodeID"]],
            border_node_ids=None,
            direction="upstream",
            split_points=nodes_left,
            order_first=True
        )
        if edges_order_all is None:
            edges_order_all = edges_order[edges_order[f"upstream_node_{outflow_node["nodeID"]}"]].copy()
        else:
            edges_order_all = pd.concat([edges_order_all, edges_order[edges_order[f"upstream_node_{outflow_node["nodeID"]}"]]])
        
        new_outflow_nodes = order.nodes[order.nodes["nodeID"].isin(new_outflow_nodes)][["nodeID", "geometry"]]
        new_outflow_nodes["orde_nr"] = order_nr
        new_outflow_nodes["rws_code"] = outflow_node["rws_code"]
        new_outflow_nodes["rws_code_no"] = outflow_node["rws_code_no"]
        new_outflow_nodes["orde_code"] = outflow_node["orde_code"]
        new_outflow_nodes["orde_nr"] = order_nr + 1
        if new_outflow_nodes_all is None:
            new_outflow_nodes_all = new_outflow_nodes.copy()
        else:
            new_outflow_nodes_all = pd.concat([new_outflow_nodes_all, new_outflow_nodes])

    edges_order = edges_order_all[edges_order_all[[f"upstream_node_{n}" for n in outflow_nodes["nodeID"]]].sum(axis=1)>0]
    edges_left = edges_left[~edges_left.code.isin(edges_order.code)].copy()
    nodes_left = nodes_left[nodes_left.nodeID.isin(edges_left.node_start)].copy()
    
    order.edges.loc[order.edges.code.isin(edges_order.code), "orde_nr"] = order_nr
    order.outflow_nodes_all = pd.concat([order.outflow_nodes_all, new_outflow_nodes_all])

    order_nr = order_nr + 1

order.generate_folium_map(base_map="Light Mode", order_labels=order_labels)
order.folium_map

In [ ]:
order.nodes.to_file(Path(order.dir_inter_results, "nodes.gpkg"))
order.edges.to_file(Path(order.dir_inter_results, "edges.gpkg"))

In [ ]:
order.generate_folium_map(base_map="Light Mode");

In [ ]:
for orde_nr in range(10):
    display(order.outflow_nodes_all[order.outflow_nodes_all["orde_nr"]==orde_nr])